In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GamingCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [2]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [3]:
df = df.dropna()
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [4]:
df.count()

4863497

In [5]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RDIJS7QYB6XNR|   18778586|B00EDBY7X8|     122952789| 2015-08-31|
|R36ED1U38IELG8|   24769659|B00D7JFOPC|     952062646| 2015-08-31|
| R1UE3RPRGCOLD|   44331596|B002LHA74O|     818126353| 2015-08-31|
|R298788GS6I901|   23310293|B00ARPLCGY|     261944918| 2015-08-31|
|  RNX4EXOBBPN5|   38745832|B00UZOPOFW|     717410439| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [8]:
review_id_df = review_id_df.dropDuplicates(["review_id"])


In [9]:
from pyspark.sql.types import * 

review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))

review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
products_df = df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EDBY7X8|Monopoly Junior B...|
|B00D7JFOPC|56 Pieces of Wood...|
|B002LHA74O|Super Jumbo Playi...|
|B00ARPLCGY|Barbie Doll and F...|
|B00UZOPOFW|Emazing Lights eL...|
+----------+--------------------+
only showing top 5 rows



In [11]:
products_df = products_df.dropDuplicates(["product_id"])


In [12]:
customers_df = df.select(["customer_id"])
customers_df.show(5)

+-----------+
|customer_id|
+-----------+
|   18778586|
|   24769659|
|   44331596|
|   23310293|
|   38745832|
+-----------+
only showing top 5 rows



In [13]:
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   49997672|    2|
|   27435626|    2|
|   50312871|   24|
|   43801871|    2|
|   33212984|    2|
+-----------+-----+
only showing top 5 rows



In [14]:
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49997672|             2|
|   27435626|             2|
|   50312871|            24|
|   43801871|             2|
|   33212984|             2|
+-----------+--------------+
only showing top 5 rows



In [15]:
customers_df = customers_df.withColumn("customer_id",customers_df["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers_df["customer_count"].cast(IntegerType()))

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [16]:
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDIJS7QYB6XNR|          5|            0|          0|   N|
|R36ED1U38IELG8|          5|            0|          0|   N|
| R1UE3RPRGCOLD|          2|            1|          1|   N|
|R298788GS6I901|          5|            0|          0|   N|
|  RNX4EXOBBPN5|          1|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [17]:
vine_df = vine_df.dropDuplicates(["review_id"])

In [18]:
vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df["total_votes"].cast(IntegerType()))

vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Write Data to Database

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cww47e97r82y.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}


In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)


In [ ]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)


In [ ]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)


In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)